# Using notebooks we will explore the process of trainign and consuming a model

First let's load some packages to maniupulate images


In [1]:
#r "nuget:SixLabors.ImageSharp,1.0.2"

## Get images

We can download images from the web, let's create some helper function

In [1]:
using SixLabors.ImageSharp;
using SixLabors.ImageSharp.PixelFormats;
using System.Net.Http;

Image GetImage(string url)
{
    var client = new HttpClient();
    var image = client.GetByteArrayAsync(url).Result;
    return Image.Load(image);
}

In [1]:
var image = GetImage("https://user-images.githubusercontent.com/2546640/56708992-deee8780-66ec-11e9-9991-eb85abb1d10a.png");
image

## it would be better to see the image, let's use the foramtter api

In [1]:
using System.IO;
using SixLabors.ImageSharp.Formats.Png;
using Microsoft.DotNet.Interactive.Formatting;

Formatter.Register<Image>((image, writer) =>
{
    var id = Guid.NewGuid().ToString("N");
    using var stream = new MemoryStream();
    image.Save(stream, new PngEncoder());
    stream.Flush();
    var data = stream.ToArray();
    var imageSource = $"data:image/png;base64, {Convert.ToBase64String(data)}";
    PocketView imgTag = PocketViewTags.img[id: id, src: imageSource, height: image.Height, width: image.Width]();
    writer.Write(imgTag);
}, HtmlFormatter.MimeType);


In [1]:
image

Good but something smaller would be better

In [1]:
using SixLabors.ImageSharp.Processing;

Image Reduce(Image source, int maxSize = 300){
    var max = Math.Max(source.Width, source.Height);
    var ratio = ((double)(maxSize)) / max;
    return source.Clone(c => c.Resize((int)(source.Width * ratio), (int)(source.Height * ratio)));
}


In [1]:
Reduce(image)

Better, now I am interested in bayblade, let's display some

In [1]:
var urls = new string[]{
    "https://cdn.shopify.com/s/files/1/0016/0674/6186/products/B154_1_1024x1024.jpg?v=1573909023",
    "https://i.ytimg.com/vi/yUH2QeluaIU/maxresdefault.jpg",
    "https://www.biggerbids.com/members/images/29371/public/8065336_-DSC5628-32467-26524-.jpg",
    "https://i.ytimg.com/vi/BT4SwVmnqqQ/maxresdefault.jpg",
    "https://cdn.shopify.com/s/files/1/0016/0674/6186/products/B160covercopy2_1200x1200.jpg?v=1585425105",
    "https://animeukiyo.com/wp-content/uploads/2020/05/king-helios-zone-1B-1140x570.jpg",
    "https://http2.mlstatic.com/beyblade-burn-phoenix-ice-blue-90wf-takara-tomy-frete-pac-D_NQ_NP_19415-MLB20171031427_092014-F.jpg"
};

var beyBlades = urls.Select(url => new { Image = Reduce(GetImage(url))});


In [1]:
beyBlades

## Enter lobe
We will now use lobe and it's .NET Bindings to developa model to classify those images. Let's start lobe and have a look first, then we will proceed with loading the pacakges we need.

In [1]:
#r "nuget:lobe"
#r "nuget:lobe.ImageSharp"

using lobe;
using lobe.ImageSharp;

Lobe can be accessed via web api let's use that for fast loops

In [1]:
#r "nuget:lobe.Http"

using lobe.Http;


In [1]:
var beyblades_start = new Uri("http://localhost:38100/predict/3af915df-14b7-4834-afbd-6615deca4e26");
var beyblades = new Uri("http://localhost:38100/predict/f56e1050-391e-4cd6-9bb9-ff74dc4d84f5");
var beyblades_2 = new Uri("http://localhost:38100/predict/f56e1050-391e-4cd6-9bb9-ff74dc4d84f5");
var beyblades_3 = new Uri("http://localhost:38100/predict/a3271b3a-f63b-4c00-9304-beda43375284");
var beyblade_remote = new Uri("http://lobe-diego.ngrok.io/predict/2a6a3005-a8cc-4bc1-a71a-a0fe85f258bb");

var httpClassifier = new LobeClient(beyblades_3);

httpClassifier.Classify(beyBlades.First().Image.CloneAs<Rgb24>())



In [1]:
var imageSources = urls.Select(url => Reduce(GetImage(url),800).CloneAs<Rgb24>()).ToList();

In [1]:
var classifications = imageSources.Select((img) => {
    var cls = httpClassifier.Classify(img);
    return new {
        Image = Reduce(img),
        Label = cls.Prediction.Label,
        Confidence = cls.Prediction.Confidence
    };
});

classifications